### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [89]:
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2021-03-19 06:01:40--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-03-19 06:01:41--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc58e305884f812060ebeb731341.dl.dropboxusercontent.com/cd/0/get/BK8Uq2anSGdYjLvA2OhTozTcQJeNcYqS0Xc5bmzLI7E_OLTdnrEcChh-nJGm3CacFZb1mZEFPjj7AkMQI2z9qRKMBhe1m4nzQUtQGrOdQyMKjHDgCWTzIQREaRXMdvj538OCs5WArR8lwrGgOq5d2cs2/file?dl=1# [following]
--2021-03-19 06:01:41--  https://uc58e305884f812060ebeb731341.dl.dropboxusercontent.com/cd/0/get/BK8Uq2anSGdYjLvA2OhTozTcQJeNcYqS0Xc5bmzLI7E_OLTdnrEcChh-nJGm3

,author,day,id,link,month,summary,tag,title,year
26879,"[{'name': 'Hardie Cate'}, {'name': 'Fahim Dalv...",7,1701.01875v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,Devices like the Myo armband available in the ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Sign Language Recognition Using Temporal Class...,2017
36729,"[{'name': 'Supratik Chakraborty'}, {'name': 'D...",11,1404.2984v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,Given a CNF formula and a weight for each assi...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Distribution-Aware Sampling and Weighted Model...,2014
7457,"[{'name': 'Mikołaj Bińkowski'}, {'name': 'Doug...",4,1801.01401v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,We investigate the training and performance of...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Demystifying MMD GANs,2018
33611,"[{'name': 'Xinsheng Lai'}, {'name': 'Yuren Zho...",3,1409.1073v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",9,Some experimental investigations have shown th...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Performance Analysis on Evolutionary Algorithm...,2014
39650,"[{'name': 'Yazhou Yao'}, {'name': 'Jian Zhang'...",22,1611.07156v4,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,Labelled image datasets have played a critical...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Exploiting Web Images for Dataset Construction...,2016


In [90]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

Let's clean the data. Use WordPunctTokenizer or something.


In [91]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(x.lower())) for x in lines]

In [92]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [93]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for x in lines:
        words = x.split()
        words.append(EOS)
        for i in range(len(words)):
            ls = [UNK] * np.maximum(0, (n - i - 1))
            begin_index = np.maximum(0, (i - n + 1))
            prefix = ls + words[begin_index:i]
            prefix = tuple(prefix)
            counts[prefix][words[i]] += 1
    
    return counts


In [94]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [95]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        denoms = defaultdict()
        for k in counts.keys():
            denoms[k] = np.sum([v for v in counts[k].values()])
        for k in counts.keys():
            for kv in counts[k].keys():
                self.probs[k][kv] = counts[k][kv] / denoms[k]
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [96]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [97]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [98]:
def get_next_token(lm: NGramLanguageModel, prefix: tuple, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    candidates = lm.get_possible_next_tokens(prefix)
    if temperature == 0.0:
        probs = [(lm.get_next_token_prob(prefix, x), x) for x in candidates]
        probs = sorted(probs, reverse=True)
        return probs[0][1]
    
    probs = [(lm.get_next_token_prob(prefix, x)**(1./temperature), x) for x in candidates]
    total_prob = np.sum([x[0] for x in probs])
    probs = [(x[0] / total_prob, x[1]) for x in probs]
    probs = sorted(probs, reverse=True)
    candidate = np.random.choice(a=[x[1] for x in probs], p=[x[0] for x in probs])
    return candidate

In [99]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [100]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial intelligence is a computer vision . this model to estimate confidence ? in which we augmented the training time . our study , demonstrating the ability to link consecutive layers of representation is crucial for actively controlled endoscopic capsule robot operations . to test an artificial limb : preliminary experiments on various metric learning ; we study a row , or run fast and accurate with respect to high - dimensional regression with convolutional and random : 20 different amino acids solutions affected by the consensus operator in matroids are investigated . this paper , we implemented dimension reduction for


In [101]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two - step of the proposed method is used to train classifiers using the proposed approach . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [102]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    result = 0
    N = 0
    n = lm.n
    for line in lines:
        words = [UNK] + line.split() + [EOS]
        for i in range(1, len(words)):
            word = words[i]
            N += 1
            prefix_str = ' '.join(words[:i])
            word_log_prob = np.maximum(min_logprob, np.log2(lm.get_next_token_prob(prefix_str, word)))
            result += word_log_prob
    ppl = 2 ** (-1 / N * result)
    return ppl

In [103]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log2


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [104]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log2


N = 1, Perplexity = 1420.16826
N = 2, Perplexity = 1031431.04445
N = 3, Perplexity = 10119238009390706.00000


In [105]:
# whoops, it just blew up :)

### Homework. LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

In [111]:
class LaplaceLanguageModel(NGramLanguageModel): 
    # your code goes here
    pass
        

In [107]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "Something is really wrong :)"

In [108]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [109]:
# optional: try to sample tokens from such a model
prefix = 'I love kittens' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

I love kittens summative ioct swing neural relus stump automatons aapm 9720 conformations lis08 asters procure mutations fecal slicings tissue rfs tax delimitation sigma_s z_b commercially functionals onto pornography gunsat alongside disentangle nmn uprisings russicum respected spinal dmn visualizing joy geosnakes evaluationnet appalachia oft analyzers xenophon rework intradomain hieroglyphs sosps crave capped footfall degeneration ubernet lawmakers imposed monochromes bssr malayo squad flavours confusion neurorobotics capacitating bounce risen gmos minutes screenernet remedy packing }|\ alterations mill pragmatically shuttlenet tangentially multidimensionality redfree decoys melodyrnn r2014a overcompleteness 943 análise discounting c5 med13 occupany 5x lrw arnetminer gatherer pregnant isaiah advntages terminations leskovec marco aforemen discours debate


In [110]:
a=5

I love kittens timesteps frmc project , we also propose a framework for the - based on the - based on the proposed method is a given the " ( i . we propose a number of the - to the proposed to the best of the solution to a new approach to the problem of the proposed in this paper , we show that may be used in this paper , and the proposed framework for the proposed a proof of the - world , we propose a novel algorithm to the loss . the - based on the input and the
